In [14]:
# with open(oname,'r') as f:
#     lines = f.readlines()
import Bio.SearchIO
import scipy.sparse
from domutil.util import *
import subprocess
import sys
did2node = pk_load('hashcath')
# tm,oms = pk_load('1200seq')




In [12]:
ds = [{1:2},{2:3},{3:4}]
# print {1:2}.update({3:4})
# reduce(lambda x,y: x + y,ds)
sum(ds,{})

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [16]:
from tempfile import TemporaryFile , mkdtemp

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def split_file(fname, number = None, linecount = None, delimiter = '\n',header = '\n',footer = '\n'):
    if linecount and number:
        raise Exception('You can only specify one of linecount or number')
    f_handles = []
    tempdir = mkdtemp(prefix = '/tmp/feng')
    
#     lcount = file_len(fname)
    with open(fname,'r') as f:
#         lines = f.readlines()
        lines = f.read().lstrip(header).rstrip(footer).split(delimiter)
        lcount = len(lines)
    print tempdir
    print lcount 
#     return
    
    if number:
#         with open(fname) as f:
#             lines = f.readlines()
        lcount = len(lines)
        nlcount = lcount // number + 1
        idxs = range(0, lcount, nlcount)
        for i,idx in enumerate(idxs):
            print i
#             if i >= 11:
#                 break

#             f = TemporaryFile()
            temp_fname = tempdir + '/%d'%i
            f = open(temp_fname, 'w+')
            f.write( header)
            f.write( delimiter.join(lines[idx: idx+nlcount]))
            f.write( footer)
#             f.close()
#             f = open(temp_fname,'r')
#             f.seek(0)
            f_handles.append(temp_fname)
    if linecount:
        idx = 0
        i   = 0
        nlcount = linecount
        while 1:
            temp_fname = tempdir + '/%d'%i
            f = open(temp_fname, 'w' )
            f.write( header)
            f.write( delimiter.join(lines[idx:idx + nlcount]) )
            f.write( footer)
            f.seek(0)
            f_handles.append(temp_fname)
            if idx + nlcount+1 >= lcount: 
                break
            else:
                idx += nlcount
                i   += 1
    return f_handles

In [44]:
import Bio.SeqIO
def hashfasta(fname):
    it = Bio.SeqIO.parse( fname,'fasta' )
    hash2did = {}
    # i = 0
    for i,ent in enumerate(it):
        h = hash(ent.name) 
        hash2did[ h ] = (i,ent.name)
    return hash2did

def f(oname):
#     it = Bio.SearchIO.parse( oname,'fasta-m10' )
#     d = {}
    it = Bio.SearchIO.parse( oname,'fasta-m10' )
    l = []
    for ent in it:
        hits = ent.__dict__['_items']
        qid = hash(ent.id)
        for hit in hits.values():
            v = hit[0].bitscore
            jid = hash(hit.id)
            l += [((qid,jid),v)]
    #         d[ ( qid, jid) ] = v
    return l
def out2dmat( f_handles, hash2did, para = 10):
# parse
    # ent = next(it)
    # ?ent
    # hits = ent.__dict__['_items']

    xs = []
    ys = []
    if para:
        pool = mp.Pool( para )
        res = pool.map(f, f_handles)
        lst = sum( res,[] )
        d = dict(lst)
        pool.close()
#         pool.join()
    else:
        d  = {}
        i = 0
        for oname in f_handles:        
            print i
            i += 1
            it = Bio.SearchIO.parse( oname,'fasta-m10' )
            for ent in it:
            #     print ent
                hits = ent.__dict__['_items']
                qid = hash(ent.id)
                for hit in hits.values():
                    v = hit[0].bitscore
                    jid = hash(hit.id)
                    d[ ( qid, jid) ] = v

    l = len(hash2did)
    om = scipy.sparse.dok_matrix( (l,l) )
    order_d = { (hash2did[k[0]][0],
                hash2did[k[1]][0]):
               v for k,v in d.iteritems()
              }
    om.update(order_d)
    return om


In [23]:
import time
import subprocess

p = subprocess.Popen(["echo", "tmp"], 
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE
                    )
# Better: p = subprocess.Popen(["sleep", "30"])

# Wait until process terminates
while p.poll() is None:
    time.sleep(0.5)

# It's done

print "Process ended, ret code:", p.returncode
res = p.communicate()
print res


Process ended, ret code: 0
('tmp\n', '')


In [22]:
f_handles = split_file( 'tmp', 10, delimiter = '>>><<<',ender = '>>>///')
# file_len('cath-domain-seqs-S35.fa')

/tmp/fengEBh8IJ
1401
0
1
2
3
4
5
6
7
8
9


In [17]:
f_handles

['/tmp/feng_3j7uW/0',
 '/tmp/feng_3j7uW/1',
 '/tmp/feng_3j7uW/2',
 '/tmp/feng_3j7uW/3',
 '/tmp/feng_3j7uW/4',
 '/tmp/feng_3j7uW/5',
 '/tmp/feng_3j7uW/6',
 '/tmp/feng_3j7uW/7',
 '/tmp/feng_3j7uW/8',
 '/tmp/feng_3j7uW/9']

In [ ]:
%%time
forig = full("./cath-domain-seqs-S35.fa")
# forig = full("$SEQlib/cath-domain-seqs-S35.fa")if ncore:
#     fs = split_file('tmp',ncore, delimiter = '>>><<<')
# else:

ftmp = forig.rstrip('.fa') + '.tmp'
fout = ftmp+'.m10'
hash2head = hashfasta(forig)
# om = out2dmat(fout,hash2head, para = 10)

ncore = 2
# ncore = 0
# if ncore:
#     fs = split_file('tmp',ncore, delimiter = '>>><<<')
# else:
#     fs = ['tmp']

header = '''
# fasta36 ./cath-domain-seqs-S35.tmp ./cath-domain-seqs-S35.tmp -T512 -m10 -f0 -
g0
FASTA searches a protein or DNA sequence data bank
 version 36.3.8d Apr, 2016(preload9)
Please cite:
 W.R. Pearson & D.J. Lipman PNAS (1988) 85:2444-2448

Query: ./cath-domain-seqs-S35.tmp
  1>>>cath|4_1_0|16pkA01/5-192 - 188 aa
Library: ./cath-domain-seqs-S35.tmp
   200787 residues in  1400 sequences

'''

fname = split_file('tmp',linecount = 60,
                   delimiter = '>>><<<',
                   header = header,
                   footer = '\n>>>///\n')[0]
# fname = 'tmp'
fs = split_file( fname, 2 , delimiter = '>>><<<',
                    header = header,
                    footer = '\n>>>///\n')
ncore = 50
fs = split_file('tmp', ncore,
                   delimiter = '>>><<<',
                   header = header,
                   footer = '\n>>>///\n')

# fs  = [fname]

# %time 
%time om = out2dmat( fs ,hash2head, para = ncore)
# oms = [om]if ncore:
#     fs = split_file('tmp',ncore, delimiter = '>>><<<')
# else:

# # om = om.todense()

#         return d

In [27]:
print file_len('tmp')
print file_len(fout)

7040319


KeyboardInterrupt: 

In [16]:

sidx = Bio.SearchIO.index('tmp', format = 'fasta-m10')
# sidx[0]
# sorted(sidx)

In [ ]:
tm = tm.tocsr()
xss = []
yss = []
for om in oms[:]:
#     d = scipy.sparse.dok_matrix( (l,l) )
#     d.update(
#         { (idxD[i],idxD[j]):v for (i,j),v in om.iteritems()}
#     )
#     om = d.tocsr()
    om = om.tocsr()
    thres = 1  
    xs = []
    ys = []
    it = np.linspace( om.min(), om.max(), 200)
    for thres in it:
        tp,fp = compare( om > thres ,tm)
        xs += [fp]
        ys += [tp] 
    xss += [xs]
    yss += [ys]

In [ ]:
# compare(tm ,om)
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure()
ax1 = plt.subplot(111)
ax1.set_xlabel('False positive')
ax1.set_ylabel('True positive')
ax1.set_xscale('log')
ax1.set_yscale('log')

lbl = 0
for xs,ys in izip(xss,yss):
    lbl += 1
    if lbl ==6:
        break
    ax1.plot(xs,ys, label = lbl)
# ax1.set_ylim(bottom = 40)
# ax1.set_ylim(bottom = 40)

ax1.legend()

# np.mean(tm.todense())